In [1]:
import minio

In [ ]:
print("hello world!")

### Spark сессия

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

drivers = [
    "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar",             # S3
    "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/home/jovyan/work/spark-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark2")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

25/11/04 15:42:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Читаем датафрейм

In [2]:
df_2023_01 = spark.read.parquet("s3a://silver/nyc-taxi-data/yellow_tripdata_2023-01")

25/11/04 15:42:42 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [ ]:
df_2024_01 = spark.read.parquet("s3a://bronze/nyc-taxi-data/yellow_tripdata_2024-01.parquet")
df_2025_01 = spark.read.parquet("s3a://bronze/nyc-taxi-data/yellow_tripdata_2025-01.parquet")

In [ ]:
layer = "silver"

df = (spark.read
      .format("parquet")
      .option("mergeSchema", "true")
      .load([
          f"s3a://{layer}/nyc-taxi-data/yellow_tripdata_2023-*",
          f"s3a://{layer}/nyc-taxi-data/yellow_tripdata_2024-*",
          f"s3a://{layer}/nyc-taxi-data/yellow_tripdata_2025-*",
      ])
     )

In [3]:
print(f"Количетсво строк: {df_2023_01.count()}\n")

[Stage 1:==================================>                       (6 + 4) / 10]

Количетсво строк: 3066766



In [ ]:
df.show(1)

In [ ]:
print(f"Количетсво строк: {df.count()}\n")

(df.withColumn("year",  F.year("tpep_pickup_datetime"))
   .withColumn("month", F.month("tpep_pickup_datetime"))
   .select("year", 
           "month", 
           "VendorID",
           "tpep_pickup_datetime", 
           "passenger_count", 
           "trip_distance",
           "airport_fee",
          )
   .groupBy("year", "month")
   .agg(
       F.count("*").alias("total_records"),
       F.sum("airport_fee").alias("Обдираловка"),
       )
   .orderBy("year", "month")
   .show(50)
)

### Исследуем

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType

def standardize_nyc_taxi_data(input_path, output_path):
    """
    Стандартизирует данные NYC Taxi:
    - Приводит все названия колонок к нижнему регистру
    - Приводит типы данных к унифицированным
    - Сохраняет в указанный путь
    """

    output_path = output_path.replace('.parquet', '')
    
    # Читаем исходные данные
    df = spark.read.parquet(input_path)
    
    # 1. Приводим все названия колонок к нижнему регистру
    for col_name in df.columns:
        df = df.withColumnRenamed(col_name, col_name.lower())
    
    # 2. Определяем маппинг типов для унификации
    type_mapping = {
        # Числовые идентификаторы - в Integer
        "vendorid": IntegerType(),
        "pulocationid": IntegerType(), 
        "dolocationid": IntegerType(),
        "payment_type": IntegerType(),
        "ratecodeid": IntegerType(),
        
        # Пассажиры - в Integer (более логично)
        "passenger_count": IntegerType(),
        
        # Денежные суммы - в Double (для точности)
        "fare_amount": DoubleType(),
        "extra": DoubleType(),
        "mta_tax": DoubleType(),
        "tip_amount": DoubleType(),
        "tolls_amount": DoubleType(),
        "improvement_surcharge": DoubleType(),
        "total_amount": DoubleType(),
        "congestion_surcharge": DoubleType(),
        "airport_fee": DoubleType(),
        "cbd_congestion_fee": DoubleType(),
        
        # Дистанция - в Double
        "trip_distance": DoubleType()
    }
    
    # 3. Применяем преобразование типов с обработкой ошибок
    for col_name, target_type in type_mapping.items():
        if col_name in df.columns:
            df = df.withColumn(
                col_name, 
                F.coalesce(
                    F.col(col_name).cast(target_type), 
                    F.lit(0 if target_type == IntegerType() else 0.0)
                )
            )
    
    # 4. Гарантируем порядок колонок для consistency
    expected_columns = [
        "vendorid", "tpep_pickup_datetime", "tpep_dropoff_datetime",
        "passenger_count", "trip_distance", "ratecodeid", "store_and_fwd_flag",
        "pulocationid", "dolocationid", "payment_type", "fare_amount", "extra",
        "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge",
        "total_amount", "congestion_surcharge", "airport_fee", "cbd_congestion_fee"
    ]
    
    # Выбираем только существующие колонки в правильном порядке
    final_columns = [col for col in expected_columns if col in df.columns]
    df_standardized = df.select(final_columns)
    
    # 5. Сохраняем с оптимальными настройками
    (df_standardized
     .coalesce(1)
     .write
     .mode("overwrite")
     .option("compression", "snappy")  # хороший баланс скорость/сжатие
     .parquet(output_path)
    )
    
    print(f"✅ Стандартизировано: {input_path} -> {output_path}")
    # print(f"📊 Схема после стандартизации:")
    # df_standardized.printSchema()
    
    return df_standardized

In [ ]:
# Пример использования для одного файла
input_path = "s3a://bronze/nyc-taxi-data/yellow_tripdata_2023-01.parquet"
output_path = "s3a://silver/nyc-taxi-data/yellow_tripdata_2023-01.parquet"

standardized_df = standardize_nyc_taxi_data(input_path, output_path)

In [ ]:
def process_all_nyc_taxi_files():
    """Обрабатывает все файлы NYC Taxi из Bronze в Silver"""
    
    # Получаем список всех parquet файлов в bronze
    bronze_files = spark.sql(f"""
        SELECT path 
        FROM (
            SELECT input_file_name() as path 
            FROM parquet.`s3a://bronze/nyc-taxi-data/`
        ) 
        GROUP BY path
    """).collect()
    
    print(f"📁 Найдено файлов для обработки: {len(bronze_files)}")
    
    for i, row in enumerate(bronze_files, 1):
        input_path = row['path']
        # Извлекаем имя файла из пути
        file_name = input_path.split('/')[-1]
        output_path = f"s3a://silver/nyc-taxi-data/{file_name}"
        
        print(f"🔄 Обрабатываю ({i}/{len(bronze_files)}): {file_name}")
        
        try:
            standardize_nyc_taxi_data(input_path, output_path)
        except Exception as e:
            print(f"❌ Ошибка при обработке {file_name}: {e}")
    
    print("🎉 Все файлы обработаны!")

In [ ]:
import time

start_time = time.time()

# Запускаем обработку всех файлов
process_all_nyc_taxi_files()

end_time = time.time()
execution_time = end_time - start_time
print(f"Время выполнения: {execution_time:.4f} секунд")

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import functions as F

def main():
    print("🚀 Starting test Spark application...")

    # Простая сессия без лишних конфигов
    spark = SparkSession.builder \
        .appName("airflow-test-app") \
        .getOrCreate()

    # Создаем тестовый датафрейм
    schema = StructType([
        StructField("id", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("value", IntegerType(), True)
    ])

    data = [
        (1, "Alice", 100),
        (2, "Bob", 200),
        (3, "Charlie", 300),
        (4, "David", 400),
        (5, "Eve", 500)
    ]

    df = spark.createDataFrame(data, schema=schema)

    print("✅ Spark session created successfully!")
    print("📊 Test DataFrame:")
    df.show()

    # Простая агрегация для демонстрации
    result = (df
              .groupBy()
              .agg(
                  F.sum("value").alias("summ"),
                  F.avg("value").alias("average"),
                  F.max("name").alias("max_name"),                 
              )
              .collect())
    total_value = result[0][0]
    print(f"💰 Total value: {total_value}")

    print(result)
    print(result[0])
    print(result[0][0])
    print(result[0][1])
    
    print("✅ Spark application completed successfully!")
    spark.stop()

if __name__ == "__main__":
    main()

🚀 Starting test Spark application...
✅ Spark session created successfully!
📊 Test DataFrame:
+---+-------+-----+
| id|   name|value|
+---+-------+-----+
|  1|  Alice|  100|
|  2|    Bob|  200|
|  3|Charlie|  300|
|  4|  David|  400|
|  5|    Eve|  500|
+---+-------+-----+

💰 Total value: 1500
[Row(summ=1500, average=300.0, max_name='Eve')]
Row(summ=1500, average=300.0, max_name='Eve')
1500
300.0
✅ Spark application completed successfully!


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType
import re
from minio import Minio
from minio.error import S3Error
import time

# Конфигурация MinIO
MINIO_ENDPOINT = 'minio:9000' 
MINIO_ACCESS_KEY = 'minioadmin'
MINIO_SECRET_KEY = 'minioadmin'


spark = SparkSession.builder \
    .appName("airflow-test-app") \
    .getOrCreate()

def get_minio_client():
    """Создает и возвращает клиент MinIO"""
    return Minio(
        MINIO_ENDPOINT,
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=False  
    )

def extract_month_from_filename(file_path):
    """Извлекает месяц из имени файла в формате YYYY-MM"""
    match = re.search(r'(\d{4}-\d{2})', file_path)
    return match.group(1) if match else None

def get_processed_slices(output_bucket, output_prefix):
    """Возвращает список уже обработанных срезов из выходного бакета используя MinIO"""
    try:
        client = get_minio_client()
        processed_slices = set()
        
        # Получаем список объектов в указанном префиксе
        objects = client.list_objects(output_bucket, prefix=output_prefix, recursive=True)
        
        for obj in objects:
            # Извлекаем месяц из пути
            month = extract_month_from_filename(obj.object_name)
            if month:
                processed_slices.add(month)
        
        print(f"📁 Найдено обработанных срезов в {output_bucket}/{output_prefix}: {len(processed_slices)}")
        return processed_slices
        
    except S3Error as e:
        if e.code == 'NoSuchBucket':
            print(f"⚠️ Бакет {output_bucket} не существует или пустой")
        else:
            print(f"⚠️ Ошибка при чтении {output_bucket} бакета: {e}")
        return set()
    except Exception as e:
        print(f"⚠️ Не удалось прочитать {output_bucket} бакет: {e}")
        return set()

def get_input_files_with_months(input_bucket, input_prefix):
    """Возвращает список файлов/папок из входного бакета с извлеченными месяцами используя MinIO"""
    try:
        client = get_minio_client()
        input_files = []
        
        # Получаем список объектов на ПЕРВОМ уровне вложенности (не рекурсивно!)
        objects = client.list_objects(input_bucket, prefix=input_prefix, recursive=False)
        
        for obj in objects:
            object_name = obj.object_name
            
            # Определяем тип объекта: файл .parquet или папка
            is_parquet_file = object_name.endswith('.parquet')
            is_folder = not is_parquet_file and object_name.endswith('/')
            
            if is_parquet_file or is_folder:
                month = extract_month_from_filename(object_name)
                if month:
                    # Формируем S3 путь для Spark
                    s3_path = f"s3a://{input_bucket}/{object_name}"
                    input_files.append({
                        'path': s3_path,
                        'month': month,
                        'file_name': object_name.split('/')[-1] if is_parquet_file else object_name.split('/')[-2] + '/'
                    })
        
        print(f"📁 Найдено объектов в {input_bucket}/{input_prefix}: {len(input_files)}")
        return input_files
        
    except Exception as e:
        print(f"❌ Ошибка при чтении {input_bucket} бакета: {e}")
        return []

def standardize_nyc_taxi_data(input_path, output_path):
    """
    Стандартизирует данные NYC Taxi (ваша существующая функция без изменений)
    """
    output_path = output_path.replace('.parquet', '')
    
    # Читаем исходные данные
    df = spark.read.parquet(input_path)
    
    # 1. Приводим все названия колонок к нижнему регистру
    for col_name in df.columns:
        df = df.withColumnRenamed(col_name, col_name.lower())
    
    # 2. Определяем маппинг типов для унификации
    type_mapping = {
        "vendorid": IntegerType(),
        "pulocationid": IntegerType(), 
        "dolocationid": IntegerType(),
        "payment_type": IntegerType(),
        "ratecodeid": IntegerType(),
        "passenger_count": IntegerType(),
        "fare_amount": DoubleType(),
        "extra": DoubleType(),
        "mta_tax": DoubleType(),
        "tip_amount": DoubleType(),
        "tolls_amount": DoubleType(),
        "improvement_surcharge": DoubleType(),
        "total_amount": DoubleType(),
        "congestion_surcharge": DoubleType(),
        "airport_fee": DoubleType(),
        "cbd_congestion_fee": DoubleType(),
        "trip_distance": DoubleType()
    }
    
    # 3. Применяем преобразование типов с обработкой ошибок
    for col_name, target_type in type_mapping.items():
        if col_name in df.columns:
            df = df.withColumn(
                col_name, 
                F.coalesce(
                    F.col(col_name).cast(target_type), 
                    F.lit(0 if target_type == IntegerType() else 0.0)
                )
            )
    
    # 4. Гарантируем порядок колонок для consistency
    expected_columns = [
        "vendorid", "tpep_pickup_datetime", "tpep_dropoff_datetime",
        "passenger_count", "trip_distance", "ratecodeid", "store_and_fwd_flag",
        "pulocationid", "dolocationid", "payment_type", "fare_amount", "extra",
        "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge",
        "total_amount", "congestion_surcharge", "airport_fee", "cbd_congestion_fee"
    ]
    
    # Выбираем только существующие колонки в правильном порядке
    final_columns = [col for col in expected_columns if col in df.columns]
    df_standardized = df.select(final_columns)
    
    # 5. Сохраняем с оптимальными настройками
    (df_standardized
     .coalesce(1)
     .write
     .mode("overwrite")
     .option("compression", "snappy")
     .parquet(output_path)
    )
    
    print(f"✅ Стандартизировано: {input_path} -> {output_path}")
    return df_standardized

def process_incremental_nyc_taxi_files(input_bucket, input_prefix, output_bucket, output_prefix):
    """Обрабатывает только новые файлы NYC Taxi из входного бакета в выходной"""
    
    # Получаем списки обработанных и доступных файлов через MinIO
    processed_slices = get_processed_slices(output_bucket, output_prefix)
    input_files = get_input_files_with_months(input_bucket, input_prefix)
    
    # Фильтруем только новые файлы
    new_files = [f for f in input_files if f['month'] not in processed_slices]
    
    print(f"📊 Статистика:")
    print(f"   - Всего во входном бакете: {len(input_files)}")
    print(f"   - Уже в выходном бакете: {len(processed_slices)}") 
    print(f"   - Новых для обработки: {len(new_files)}")
    
    if not new_files:
        print("🎉 Все срезы уже обработаны! Ничего делать не нужно.")
        return
    
    # Обрабатываем только новые файлы
    for i, file_info in enumerate(new_files, 1):
        input_path = file_info['path']
        file_name = file_info['file_name']
        
        # Формируем выходной путь, сохраняя структуру после префикса
        # Пример: входной путь s3a://bronze/nyc-taxi-data/yellow_tripdata_2022-01
        # Выходной путь: s3a://silver/nyc-taxi-data/yellow_tripdata_2022-01
        output_path = f"s3a://{output_bucket}/{output_prefix}{file_name}".replace('.parquet', '')
        
        print(f"🔄 Обрабатываю новый срез ({i}/{len(new_files)}): {file_info['month']}")
        
        try:
            standardize_nyc_taxi_data(input_path, output_path)
            print(f"✅ Успешно обработан: {file_info['month']}")
        except Exception as e:
            print(f"❌ Ошибка при обработке {file_info['month']}: {e}")
    
    print(f"🎉 Обработка завершена! Обработано {len(new_files)} новых срезов.")

# Дополнительная функция для отладки - просмотр структуры бакетов
def inspect_buckets(bucket_name, prefix):
    """Функция для отладки - показывает структуру бакетов"""
    try:
        client = get_minio_client()
        
        print(f"🔍 Инспекция бакета {bucket_name}/{prefix}:")
        objects = client.list_objects(bucket_name, prefix=prefix, recursive=True)
        for obj in objects:
            print(f"   {bucket_name}: {obj.object_name}")
            
    except Exception as e:
        print(f"❌ Ошибка при инспекции бакета {bucket_name}: {e}")


# Обработка данных из bronze в silver
process_incremental_nyc_taxi_files(
    input_bucket='bronze', 
    input_prefix='nyc-taxi-data/',
    output_bucket='silver', 
    output_prefix='nyc-taxi-data-norm/'
)